In [60]:
import torch

import wget as wget
from tqdm.notebook import tqdm


from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import os

!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

print("PyTorch has version {}".format(torch.__version__))

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
PyTorch has version 1.13.1


In [2]:
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
#!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
#!pip install torch-geometric
#!pip install ogb
#!pip install wget

In [59]:


def get_graphs(url, folder):
  """
  Inspired by: https://python.plainenglish.io/notesdownloader-use-web-scraping-to-download-all-pdfs-with-python-511ea9f55e48
  """
  if not os.path.exists(folder):
    os.makedirs(folder)

  links = []
  html = urlopen(url).read()
  html_page = bs(html, features="lxml") 

  for link in html_page.find_all('a'):
    current_link = link.get('href')

    if current_link.endswith('gz'):
      links.append(url + current_link)

  for link in tqdm(links):
    try:
      wget.download(link, out = folder)
    except:
      print("Unable to Download A File")
  print("File download done!")

In [4]:
import gzip
import shutil
from pathlib import Path


def uncompress_files(path, outpath):
  """
  Inspired by https://stackoverflow.com/questions/3548673/how-can-i-replace-or-strip-an-extension-from-a-filename-in-python
  and https://stackoverflow.com/questions/31028815/how-to-unzip-gz-file-using-python
  """
  if not os.path.exists(outpath):
    os.makedirs(outpath)
  for file in tqdm(os.listdir(path)):
    if os.path.isdir(os.path.join(path, file)):
      continue
    new_filename = Path(file)
    extensions = "".join(new_filename.suffixes)
    new_ext = ".txt"
    new_filename = str(new_filename).replace(extensions, new_ext)
    new_filename = os.path.join(outpath, new_filename)

    with gzip.open(os.path.join(path, file), 'rb') as f_in:
        with open(new_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [6]:
get_graphs("http://lime.cs.elte.hu/~kpeter/data/mcf/road/", './data/road/compressed')

  0%|          | 0/70 [00:00<?, ?it/s]

Unable to Download A File


In [5]:
uncompress_files('./data/road/compressed', './resources')

  0%|          | 0/40 [00:00<?, ?it/s]

In [61]:
import os.path as osp

import torch
from torch_geometric.data import Data, Dataset
from data_parsing import process_file


class MinCostDataset(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        """If these files are found in the raw directory, download is skipped"""
        return []

    @property
    def processed_file_names(self):
        """If these files are found in the processed directory, processing is skipped"""
        processed_files = []
        path = self.processed_dir
        for file in tqdm(os.listdir(path)):
            file_path = os.path.join(path, file)
            if not os.path.isdir(file_path) and not file == "pre_filter.pt" and not file == "pre_transform.pt":
                processed_files.append(file)

        return processed_files

    def download(self):
        pass

    def process(self):
        idx = 0
        path = self.raw_dir
        for file in tqdm(os.listdir(path)):
            print(file)
            file_path = os.path.join(path, file)
            if os.path.isdir(file_path):
                continue
            # Read data from `raw_path`.
            output = process_file(file_path, "cbn")
            if output["converged"]:
                x = output["x"]
                edge_index = output["edge_index"]
                edge_attr = output["edge_attr"]
                y = output["y"]
                data = Data(x = x, edge_index = edge_index, edge_attr = edge_attr, y = y, filename = file_path)

                torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                idx += 1

    def len(self):
        return len(self.processed_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data

In [62]:
dataset = MinCostDataset(root = "./data/")

  0%|          | 0/134 [00:00<?, ?it/s]

In [63]:
print(dataset)

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

MinCostDataset(132)


In [64]:
def i_messed_up(dataset, idx):
    data = dataset.get(idx)
    #data.edge_index = torch.reshape(data.edge_index, (-1, 2)).T
    #data.edge_attr = torch.reshape(data.edge_attr, (-1, 2))
    #data.x = torch.reshape(data.x, (-1,1))
    # data.x = data.x.type(torch.float32)
    torch.save(data, osp.join(dataset.processed_dir, f'data_{idx}.pt'))
for i in range(dataset.len()):
    i_messed_up(dataset, i)

  0%|          | 0/134 [00:00<?, ?it/s]

In [65]:
dataset[0].edge_index

  0%|          | 0/134 [00:00<?, ?it/s]

tensor([[  16,   17,   18,  ..., 9556, 9556, 9558],
        [  17,   16,   16,  ..., 9557, 9558, 9556]])

In [72]:
import importlib
import data_parsing
importlib.reload(data_parsing)
processed = data_parsing.process_file("./data/raw/road_paths_01_DC_a.txt", "cbn")

In [75]:
processed["edge_index"]

tensor([[  17,   18,   19,  ..., 9556, 9556, 9558],
        [  18,   17,   17,  ..., 9557, 9558, 9556]])

In [74]:
processed["edge_attr"]

tensor([[  1.,  42.],
        [  1.,  42.],
        [  1.,  60.],
        ...,
        [  1., 452.],
        [  1., 231.],
        [  1., 231.]])